# V2 Model Server (SKLearn)

In [1]:
import mlrun

> 2020-10-28 16:59:31,518 [warning] Failed resolving version info. Ignoring and using defaults


### Configuration and package dependencies

In [2]:
%nuclio config kind="serving"
%nuclio config spec.build.baseImage="mlrun/ml-models"
%nuclio config spec.maxReplicas = 1

%nuclio: setting kind to 'serving'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'
%nuclio: setting spec.maxReplicas to 1


### Serving class code

In [3]:
from cloudpickle import load
from typing import List
from sklearn.datasets import load_iris
import numpy as np
import mlrun

import warnings 
warnings.filterwarnings('ignore')

In [4]:
class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [5]:
# nuclio: end-code

# Convert to function object

### Document and save (as a template)

In [6]:
fn = mlrun.code_to_function('v2-model-server', 
                            description="generic sklearn model server",
                            categories=['serving', 'ml'],
                            labels={'author': 'yaronh', 'framework': 'sklearn'},
                            code_output='.')

fn.spec.default_class = 'ClassifierModel'
#print(fn.to_yaml())
fn.export()

> 2020-10-28 16:59:40,197 [info] function spec saved to path: function.yaml


### Configure and add model(s)

In [7]:
models_path = 'https://s3.wasabisys.com/iguazio/models/iris/model.pkl'
fn.add_model('mymodel', model_path=models_path)
#fn.verbose = True

# Test models locally (using a server emulator)

In [8]:
# create an emulator (mock server) from the function configuration)
server = fn.to_mock_server(globals())

{'routes': <mlrun.model.ObjectDict object at 0x7f5cf1d741d0>}
{'model_path': 'https://s3.wasabisys.com/iguazio/models/iris/model.pkl'}
> 2020-10-28 16:59:40,542 [info] model mymodel was loaded
> 2020-10-28 16:59:40,543 [info] Loaded ['mymodel']


### Test against the iris dataset 

In [9]:
iris = load_iris()
x = iris['data'].tolist()

In [10]:
result = server.test("/v2/models/mymodel/infer", {"inputs": x})
result.keys()

> 2020-10-28 16:59:40,558 [debug] router run model mymodel, op=infer


dict_keys(['id', 'model_name', 'outputs'])

##  Deploy server

In [11]:
fn.apply(mlrun.mount_v3io())

In [12]:
fn.deploy()

> 2020-10-28 16:59:40,610 [info] deploy started
[nuclio] 2020-10-28 16:59:45,872 (info) Build complete
[nuclio] 2020-10-28 16:59:52,944 done updating default-v2-model-server, function address: default-tenant.app.dsteam.iguazio-cd1.com:30984
> 2020-10-28 16:59:52,950 [warning] warning!, server (0.5.3-rc1) and client (unstable) ver dont match


'http://default-tenant.app.dsteam.iguazio-cd1.com:30984'

##  Test server

In [13]:
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''
fn.invoke('/v2/models/mymodel/infer', my_data)

OSError: error: cannot run function at url http://default-tenant.app.dsteam.iguazio-cd1.com:30984/v2/models/mymodel/infer, HTTPConnectionPool(host='default-tenant.app.dsteam.iguazio-cd1.com', port=30984): Max retries exceeded with url: /v2/models/mymodel/infer (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f5cf14da3d0>: Failed to establish a new connection: [Errno 113] No route to host'))